In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 6.6 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
import numpy as np
import math
import operator
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from nltk.sentiment import SentimentIntensityAnalyzer
import torch
from transformers import AutoTokenizer, AutoModel

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
df = pd.read_csv('/content/Data_from_Meta.xlsx - Sheet1.csv')
df.head()

,Unnamed: 0,text,class
0,1.0,"Tik Tok hit song, I don't like ovar smart",suicide
1,2.0,Men have emotions too so don't be shy to cry. ...,suicide
2,3.0,"locked profile, bio I was little for the sake ...",suicide
3,4.0,On the occasion of 'World Suicide Prevention D...,non-suicide
4,5.0,"Endless pain, lonely nights, parents don’t eve...",suicide


In [ ]:
# Preprocess the text data by removing stop words, punctuations, and special characters
nltk.download("stopwords")
stopwords = nltk.corpus.stopwords.words("english")
df["text"] = df["text"].apply(lambda x: " ".join([word for word in x.split() if word not in stopwords]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
LE = LabelEncoder()
df['label'] = LE.fit_transform(df['class'])
df.head()

,Unnamed: 0,text,class,label
0,1.0,"Tik Tok hit song, I like ovar smart",suicide,1
1,2.0,Men emotions shy cry. It's okay let hold insid...,suicide,1
2,3.0,"locked profile, bio I little sake born growing...",suicide,1
3,4.0,"On occasion 'World Suicide Prevention Day', I ...",non-suicide,0
4,5.0,"Endless pain, lonely nights, parents don’t eve...",suicide,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df["label"], test_size=0.2, random_state=42)

In [ ]:
X_train.head()

79     In used replay situations loop. Especially thi...
197    Your suicidal friend told struggling asked why...
38               Marrying Someone Can Stop Depression???
24     What depression really like: • Not showering r...
122    mental health suffered drastically. Over years...
Name: text, dtype: object

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased").to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized_train = tokenizer(X_train.values.tolist(), padding = True, truncation = True, return_tensors="pt")
tokenized_test = tokenizer(X_test.values.tolist() , padding = True, truncation = True,  return_tensors="pt")

print(tokenized_train.keys())

tokenized_train = {k:torch.tensor(v).to(device) for k,v in tokenized_train.items()}
tokenized_test = {k:torch.tensor(v).to(device) for k,v in tokenized_test.items()}

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


<ipython-input-9-85833f1dc3b0>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_train = {k:torch.tensor(v).to(device) for k,v in tokenized_train.items()}
<ipython-input-9-85833f1dc3b0>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_test = {k:torch.tensor(v).to(device) for k,v in tokenized_test.items()}


In [ ]:
# Download the lexicon
nltk.download("vader_lexicon")
# Use VADER to extract the sentiment scores for each text
vader = SentimentIntensityAnalyzer()
vader_scores_train = [vader.polarity_scores(text) for text in X_train]
vader_scores_test = [vader.polarity_scores(text) for text in X_test]

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
vader_scores_l_test = []
for i in vader_scores_test:
    vader_scores_l_test.append(list(i.values()))

In [ ]:
vader_scores_l_test = torch.as_tensor(vader_scores_l_test)

In [ ]:
vader_scores_l_train = []
for i in vader_scores_train:
    vader_scores_l_train.append(list(i.values()))

In [ ]:
vader_scores_l_train = torch.as_tensor(vader_scores_l_train)

In [ ]:
with torch.no_grad():
  hidden_train = model(**tokenized_train) 
  hidden_test = model(**tokenized_test)


cls_train = hidden_train.last_hidden_state[:,0,:]
cls_test = hidden_test.last_hidden_state[:,0,:]

In [ ]:
X_train = cls_train.to("cpu")
X_train = torch.cat((X_train, vader_scores_l_train), dim=1)
X_test = cls_test.to("cpu")
X_test = torch.cat((X_test, vader_scores_l_test), dim=1)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch.Size([160, 772]) (160,)
torch.Size([40, 772]) (40,)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train,y_train)
rf.score(X_test,y_test) 

0.825

In [ ]:
# Use the trained classifier to predict the sentiment of the text in the testing set
y_pred = rf.predict(X_test)

In [ ]:
# Evaluate the performance of the model using metrics such as accuracy, precision, recall, and F1-score
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.42      0.59        12
           1       0.80      1.00      0.89        28

    accuracy                           0.82        40
   macro avg       0.90      0.71      0.74        40
weighted avg       0.86      0.82      0.80        40

